<a href="https://colab.research.google.com/github/waynexu2000/AlphaGoDeepMind/blob/master/DataSource.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Datasource_Catalog.xlsx to Datasource_Catalog.xlsx


In [ ]:
import pandas as pd

In [ ]:
sourcefile_name = 'Datasource_Catalog.xlsx'
sheetname = 'Datasources'
df = pd.read_excel(sourcefile_name, sheet_name = sheetname)
df['CNT']=1

In [ ]:
df.head(3)

,Line of Business,Domain,Sub Domains,Legal Entity,Source System,Current /\n Historical,Data Availability from 1/1/2016,Core Integration Patterns,Sub Integration patterns,Engineering Options,Notes,Questions / Open Items,UHC Ancillary?,DB Technology,System of Record,Source Solution Architect,Source Delivery Lead,Source Data Mapping SME,Last Updated / Reviewed,Integration Pattern -old,IN Scope,CNT
0,Direct Business,Claims,Behavioural Health,Allways - Medicaid,Optum BH - OHBS Facets,NaN,NaN,not in scope,not in scope,NaN,In scope for Optum's Interoperability Program,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,1
1,Medicare,Provider Directory,Telehealth,Rocky Mountain Health Plans,American Well Telehealth,Future,Current Data,PDP,PDP Integration Pattern - TBD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,ID/PDP solution - Integration Pattern TBD,NaN,1
2,Medicare,Provider Directory,Catalog Over the Counter,Rocky Mountain Health Plans,Firstline Medical,Future,Current Data,PDP,PDP Integration Pattern - TBD,NaN,possibly live by 1./1/2020\nCatalog OTC vendor...,NaN,NaN,NaN,NaN,"Prashanth Narasimhamurthy , John Dickey",Glen McDaniel,"Debbie Baum, Lisa Hall",NaT,ID/PDP solution - Integration Pattern TBD,NaN,1


In [ ]:
#Remove repeated association elements
df.pop("Line of Business")
df.pop("Sub Domains")

0            Behavioural Health
1                    Telehealth
2      Catalog Over the Counter
3          Post discharge meals
4            Behavioural Health
                 ...           
340                         NaN
341                         NaN
342                         NaN
343                         NaN
344                    Lab Data
Name: Sub Domains, Length: 345, dtype: object

In [ ]:
#Dedup first
dedup = df.drop_duplicates()
dedup.count()['CNT']

241

In [ ]:
dedup

In [ ]:
#dedup.columns  -- Column name "IN Scope"
InScopeFilter = (dedup['IN Scope']!='No')

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [ ]:
res = dedup.loc[InScopeFilter, dedup.columns]
res.count()['CNT']

241

In [ ]:
filterClaim = (res['Domain']=='Claims')

In [ ]:
res.loc[filterClaim, res.columns]

In [ ]:
#Count/group-by
IntePattern = res.groupby(['Core Integration Patterns','Sub Integration patterns']).count()['CNT']
IntePattern

In [ ]:
#Count/group-by  Domain= Claims
ClaimPattern = res.loc[filterClaim].groupby(['Core Integration Patterns','Sub Integration patterns']).count()['CNT']
ClaimPattern

Core Integration Patterns  Sub Integration patterns                                    
HCP                        File Exposed by Third Party in Canonical Form                   21
                           File Exposed by Third Party in Non Canonical Form               19
                           Source Streams to HCP-Core                                       1
                           Source Streams to HCP-Core                                      23
                           Source Streams to HCP-Core  (To be confirmed with OHBS Team)     1
                           Source Streams to HCP-Core - Will need integration support      13
                           Source Streams to HCP-Hemi (Rx Claims only)                      8
                           not in scope                                                     3
not in scope               not in scope                                                    16
not in scope.              not in scope                           

In [ ]:
filterElig = (res['Domain']=='Eligibility')

In [ ]:
#Count/group-by  Domain= Eligibility
EligPattern = res.loc[filterElig].groupby(['Core Integration Patterns','Sub Integration patterns']).count()['CNT']
EligPattern

Core Integration Patterns  Sub Integration patterns                         
HCP                        File Exposed by Third Party in Canonical Form        3
                           File Exposed by Third Party in Non Canonical Form    2
                           Source ETL to HCP-Core                               2
                           Source Streams to HCP-Core                           4
not in scope               not in scope                                         1
Name: CNT, dtype: int64

In [ ]:
DomainCNT = res.groupby(['Domain']).count()['CNT']
DomainCNT 

Domain
Benefits                8
Claims                111
Clinical               17
Eligibility            12
Formulary              12
Provider Directory     80
Name: CNT, dtype: int64

# **Writing to multiple sheets of an Excel file**

In [ ]:
#Writing results to multiple sheets in an Excel Workbook, with formatting
writer = pd.ExcelWriter('Datasource_Output.xlsx')

In [ ]:
files.download('Datasource_Output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
IntePattern.to_excel(writer, sheet_name = 'HCP_Inte_Pattern')
#writer.sheets['HCP_Inte_Pattern'].set_column(0,1,50)
#writer.sheets['HCP_Inte_Pattern'].write_comment('C1', 'This is the count by Integration Patterns & Sub-Patterns')

workbook =writer.book
worksheet=writer.sheets['HCP_Inte_Pattern']

#writer.sheets['HCP_Inte_Pattern'].set_column(3,4,20)

DomainCNT.to_excel(writer, sheet_name = 'DomainCount')
#writer.sheets['DomainCount'].set_column(0,0,30)
ClaimPattern.to_excel(writer, sheet_name = 'ClaimIntePattern' )
#writer.sheets['ClaimIntePattern'].set_column(0,1,50)
EligPattern.to_excel(writer, sheet_name = 'EligIntePattern' )
#writer.sheets['EligIntePattern'].set_column(0,1,50)
writer.save()